### Import the packages

In [121]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.optimizers import Adam
import math

### Define the data features

In [122]:
train_data_dir = 'data/train'
validation_data_dir = 'data/val'
train_samples = 2000
validation_samples = 1000
num_classes = 2
img_width, img_height = 224, 224
batch_size = 64

### Apply rotations, shifts and zooms on the images for data augmentation 

In [123]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input
                                  , rotation_range=20
                                  , width_shift_range=0.2
                                  , height_shift_range=0.2
                                  , zoom_range=0.2)
val_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

### Load the data and create the generators

In [124]:
train_generator = train_datagen.flow_from_directory(
                        train_data_dir
                        , target_size=(img_width, img_height)
                        , batch_size=batch_size
                        , shuffle=True
                        , seed=12345
                        , class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
                        validation_data_dir
                        , target_size=(img_width, img_height)
                        , batch_size=batch_size
                        , shuffle=True
                        , seed=12345
                        , class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Create a function to fit the model to our data

In [125]:
def model_maker():
    base_model = MobileNet(include_top=False, input_shape=(img_width, img_height, 3)) #Use the MobileNet pre-trained model 
    for layer in base_model.layers[:]:
        layer.trainable = False #Freeze the layers
    input = Input(shape=(img_width, img_height, 3))
    custom_model = base_model(input) #Base model we'll use for transfer
    custom_model = GlobalAveragePooling2D() (custom_model) #Average features per categories
    custom_model = Dense(64, activation='relu') (custom_model) #NN layer
    custom_model = Dropout(0.5) (custom_model) #Layer to prevent overfitting
    predictions = Dense(num_classes, activation='softmax') (custom_model) #Final model predictions
    return Model(inputs=input, outputs=predictions)

### Apply the function to the data and fit the model to the data

In [126]:
model = model_maker()
model.compile(loss = 'categorical_crossentropy'
              , optimizer = Adam(learning_rate=0.001)
              , metrics = ['acc'])

n_steps = math.ceil(float(train_samples)/batch_size)
model.fit_generator(train_generator
                    , steps_per_epoch = n_steps
                    , epochs = 20
                    , validation_data = validation_generator
                    , validation_steps = n_steps)

/tmp/ipykernel_41260/4156942998.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,


Epoch 1/20
32/32 [==============================] - 84s 3s/step - loss: 0.3339 - acc: 0.8550 - val_loss: 0.0772 - val_acc: 0.9700
Epoch 2/20
32/32 [==============================] - 54s 2s/step - loss: 0.1142 - acc: 0.9540
Epoch 3/20
32/32 [==============================] - 55s 2s/step - loss: 0.1016 - acc: 0.9620
Epoch 4/20
32/32 [==============================] - 54s 2s/step - loss: 0.0767 - acc: 0.9705
Epoch 5/20
32/32 [==============================] - 54s 2s/step - loss: 0.0744 - acc: 0.9725
Epoch 6/20
32/32 [==============================] - 58s 2s/step - loss: 0.0681 - acc: 0.9710
Epoch 7/20
32/32 [==============================] - 55s 2s/step - loss: 0.0648 - acc: 0.9795
Epoch 8/20
32/32 [==============================] - 54s 2s/step - loss: 0.0561 - acc: 0.9770
Epoch 9/20
32/32 [==============================] - 56s 2s/step - loss: 0.0721 - acc: 0.9720
Epoch 10/20
32/32 [==============================] - 54s 2s/step - loss: 0.0636 - acc: 0.9780
Epoch 11/20
32/32 [=============

Here we see that the at the end of training the model has achieved a 98.8% accuracy on the validation set

### Save the model for easier use in the future

In [127]:
model.save('model.h5')

### Load the model and test on some images

In [128]:
model = load_model('model.h5')

In [129]:
img = image.load_img('data/dog.193.jpg', target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
prediction = model.predict(preprocessed_img)
print(prediction[0])
if np.argmax(prediction[0]) == 0:
    print('cat')
else:
    print('dog')

[3.0203511e-05 9.9996984e-01]
dog


The two images below not labeled as cat are photos of my own cat

In [130]:
img = image.load_img('chell.jpeg', target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
prediction = model.predict(preprocessed_img)
print(prediction[0])
if np.argmax(prediction[0]) == 0:
    print('cat')
else:
    print('dog')

[0.8296348 0.1703652]
cat


In [131]:
img = image.load_img('chell2.jpeg', target_size=(224,224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = preprocess_input(expanded_img_array)
prediction = model.predict(preprocessed_img)
print(prediction[0])
if np.argmax(prediction[0]) == 0:
    print('cat')
else:
    print('dog')

[9.9999976e-01 2.7523157e-07]
cat


### Test the model with a few more images to get an estimated accuracy

In [132]:
test_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [133]:
test_generator = test_datagen.flow_from_directory(
                         'data/test'
                        , target_size = (224, 224)
                        , batch_size = 64
                        , shuffle = True
                        , seed = 12345
                        , class_mode = 'categorical')

Found 1000 images belonging to 2 classes.


In [134]:
model.evaluate(test_generator)

16/16 [==============================] - 21s 1s/step - loss: 0.0567 - acc: 0.9830


[0.05665692314505577, 0.9829999804496765]

Evaluating the model on 1000 test images we see that it has a 98.3% accuracy on unseen data